## Import libraries

In [2]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm


## Load images and labels

In [3]:
# Function to load images from a folder and convert them to a tensor
def load_images(folder_path):
    images_paths = os.listdir(folder_path)
    images = []
    for filename in tqdm(images_paths, desc = "Reading images"):
        if filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert("RGB")
            img_array = np.array(img)
            images.append(img_array)

    images_tensor = torch.Tensor(images)
    return images_tensor

# Function to load CSV file and convert labels to a tensor
def load_labels(csv_path):
    df = pd.read_csv(csv_path)
    labels = df['Class'].values
    labels_tensor = torch.Tensor(labels)
    return labels_tensor


In [4]:
# Specify the paths
train_images_folder_path = "train_images"

# Load images and labels into tensors
train_images = load_images(train_images_folder_path)

# Print the shapes of the resulting tensors
print("Images Tensor Shape:", train_images.shape)


Reading images: 100%|██████████| 13000/13000 [00:14<00:00, 907.61it/s]


Images Tensor Shape: torch.Size([13000, 64, 64, 3])


C:\Users\aledragomir\AppData\Local\Temp\ipykernel_26780\2685528804.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  images_tensor = torch.Tensor(images)


In [5]:
csv_path = "train.csv"

train_labels = load_labels(csv_path)

print("Labels Tensor Shape:", train_labels.shape)


Labels Tensor Shape: torch.Size([13000])


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def SimpleCNN():
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten before fully connected layersa
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(100, activation='softmax'))  # Output size is the number of classes

    return model

# Instantiate the model
model = SimpleCNN()

# Print the model summary
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)             

In [ ]:
%pip install tensorflow

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have images_tensor and labels_tensor from previous steps

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Convert NumPy arrays to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Create DataLoader for training and testing
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# batch_size = 64
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Assuming labels are integers
              metrics=['accuracy'])

In [14]:
# Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model[0].parameters(), lr=0.001)

model.fit(X_train_tensor, y_train_tensor, epochs=10, validation_data=(X_test_tensor, y_test_tensor))



Epoch 1/10


325/325 [==============================] - 25s 73ms/step - loss: 6.8620 - accuracy: 0.0079 - val_loss: 4.6089 - val_accuracy: 0.0081
Epoch 2/10
325/325 [==============================] - 22s 69ms/step - loss: 4.5464 - accuracy: 0.0248 - val_loss: 4.6422 - val_accuracy: 0.0069
Epoch 3/10
325/325 [==============================] - 21s 65ms/step - loss: 4.2308 - accuracy: 0.0844 - val_loss: 4.9469 - val_accuracy: 0.0092
Epoch 4/10
325/325 [==============================] - 22s 68ms/step - loss: 3.6482 - accuracy: 0.2062 - val_loss: 5.9100 - val_accuracy: 0.0131
Epoch 5/10
325/325 [==============================] - 21s 64ms/step - loss: 2.9824 - accuracy: 0.3438 - val_loss: 6.9850 - val_accuracy: 0.0104
Epoch 6/10
325/325 [==============================] - 21s 65ms/step - loss: 2.4620 - accuracy: 0.4594 - val_loss: 9.1809 - val_accuracy: 0.0096
Epoch 7/10
325/325 [==============================] - 22s 69ms/step - loss: 2.1703 - accuracy: 0.5250 - val_loss: 10.7736 - val_accura